In [ ]:
import tensorflow as tf

# Check GPU availability
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ModuleNotFoundError: No module named 'numpy'

In [ ]:
# loading number data
def load_data_from_folders_number(base_dir):
    data = []
    labels = []
    for label_folder in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, label_folder)
        if os.path.isdir(folder_path):
            for file_name in os.listdir(folder_path):
                if file_name.endswith('.npy'):
                    file_path = os.path.join(folder_path, file_name)
                    landmarks = np.load(file_path)
                    data.append(landmarks)
                    labels.append(label_folder)
    return np.array(data), np.array(labels)

# Load single-hand data
number_train_data, number_train_labels = load_data_from_folders_number('/content/drive/My Drive/Colab Notebooks/ISLtoText/dataset_numbers/train')
number_test_data, number_test_labels = load_data_from_folders_number('/content/drive/My Drive/Colab Notebooks/ISLtoText/dataset_numbers/test')

# Encode labels for single-hand model
label_encoder_single = LabelEncoder()
number_train_labels_encoded = label_encoder_single.fit_transform(number_train_labels)
number_test_labels_encoded = label_encoder_single.transform(number_test_labels)

# Convert labels to one-hot encoding
number_train_labels_categorical = to_categorical(number_train_labels_encoded)
number_test_labels_categorical = to_categorical(number_test_labels_encoded)

In [ ]:
class PrintLossCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        train_loss = logs.get('loss')
        val_loss = logs.get('val_loss')
        print(f"Epoch {epoch + 1}: Training Loss = {train_loss}, Validation Loss = {val_loss}")

# Instantiate the callback
print_loss_callback = PrintLossCallback()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define a callback to save model checkpoints
checkpoint_callback_number = ModelCheckpoint(
    '/content/drive/My Drive/Colab Notebooks/ISLtoText/number_model_checkpoint.keras',    # Correct file path extension
    save_best_only=True,               # Save only the best weights
    monitor='val_loss',               # Monitor validation loss to save the best model
    mode='min'                        # Mode 'min' for minimization
)

In [ ]:
def create_number_model(input_shape, num_classes):
    model = Sequential([
        Dense(128, activation='relu', input_shape=input_shape),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model
# Define and train the model for single-hand signs
number_model = create_number_model((63,), len(label_encoder_single.classes_))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/content/drive/My Drive/Colab Notebooks/ISLtoText/number_model.keras')

#model = load_model('/content/drive/My Drive/Colab Notebooks/ISLtoText/number_model_checkpoint.keras')

: 

In [ ]:
number_model.fit(
    number_train_data,
    number_train_labels_categorical,
    epochs=20,
    batch_size=32,
    validation_data=(number_test_data, number_test_labels_categorical),
    callbacks=[print_loss_callback, checkpoint_callback_number]
    )

Epoch 1/20
179/183 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1478 - loss: 2.2010Epoch 1: Training Loss = 2.083879232406616, Validation Loss = 1.7009843587875366
183/183 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.1496 - loss: 2.1979 - val_accuracy: 0.7239 - val_loss: 1.7010
Epoch 2/20
160/183 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4330 - loss: 1.6234Epoch 2: Training Loss = 1.4246199131011963, Validation Loss = 0.8906363248825073
183/183 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4389 - loss: 1.5999 - val_accuracy: 0.9106 - val_loss: 0.8906
Epoch 3/20
167/183 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6035 - loss: 1.0339Epoch 3: Training Loss = 0.9459347724914551, Validation Loss = 0.5754750370979309
183/183 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6063 - loss: 1.0262 - val_accuracy: 0.9381 - val_loss: 0.5755
Epoch 4/20
171/183 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7362 - loss: 0.7259Epoch 4: Training Loss = 0.6722191572189331, Validation Loss = 0

In [ ]:
# Save the model
number_model.save('/content/drive/My Drive/Colab Notebooks/ISLtoText/number_model.keras')

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Get predictions
y_pred = model.predict(validation_data)

# If the output is probabilistic (softmax), convert it to class labels
y_pred_classes = np.argmax(y_pred, axis=1)

# Get the true labels from the validation data
y_true = np.argmax(number_test_labels_categorical, axis=1)

conf_matrix = confusion_matrix(y_true, y_pred_classes)
print("Confusion Matrix:")
print(conf_matrix)
